<a href="https://colab.research.google.com/github/TheKeiser/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/Inlamning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [2]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
!pip install wandb
import wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 53.6 MB/s 
     |████████████████████████████████| 168 kB 40.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 52.0 MB/s 
     |████████████████████████████████| 166 kB 51.0 MB/s 
     |████████████████████████████████| 166 kB 41.3 MB/s 
     |████████████████████████████████| 162 kB 63.1 MB/s 
     |████████████████████████████████| 162 kB 59.4 MB/s 
     |████████████████████████████████| 158 kB 52.7 MB/s 
     |████████████████████████████████| 157 kB 42.1 MB/s 
     |████████████████████████████████| 157 kB 49.1 MB/s 
     |████████████████████████████████| 157 kB 65.2 MB/s 
     |████████████████████████████████| 157 kB 55.8 MB/s 
     |████████████████████████████████| 157 kB 54.0 MB/s 
     |████████████████████████████

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [3]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [4]:
def get_mnist_data(): #fashion data installed
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [5]:
batch_size = 128
epochs = 5
wandb.init(project="ai-ml-exercise", config={"batch_size": batch_size, "epochs": epochs})

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
422/422 [==============================] - ETA: 0s - loss: 0.6163 - accuracy: 0.7821

wandb: Adding directory to artifact (/content/wandb/run-20221211_101824-xt0ej000/files/model-best)... Done. 0.0s


422/422 [==============================] - 54s 126ms/step - loss: 0.6163 - accuracy: 0.7821 - val_loss: 0.4271 - val_accuracy: 0.8450
Epoch 2/5
422/422 [==============================] - ETA: 0s - loss: 0.3897 - accuracy: 0.8619

wandb: Adding directory to artifact (/content/wandb/run-20221211_101824-xt0ej000/files/model-best)... Done. 0.0s


...

422/422 [==============================] - 51s 121ms/step - loss: 0.3459 - accuracy: 0.8775 - val_loss: 0.3362 - val_accuracy: 0.8798
Epoch 4/5
422/422 [==============================] - ETA: 0s - loss: 0.3193 - accuracy: 0.8865

wandb: Adding directory to artifact (/content/wandb/run-20221211_101824-xt0ej000/files/model-best)... Done. 0.0s


422/422 [==============================] - 51s 120ms/step - loss: 0.3193 - accuracy: 0.8865 - val_loss: 0.3176 - val_accuracy: 0.8855
Epoch 5/5
422/422 [==============================] - ETA: 0s - loss: 0.2964 - accuracy: 0.8939

wandb: Adding directory to artifact (/content/wandb/run-20221211_101824-xt0ej000/files/model-best)... Done. 0.0s


422/422 [==============================] - 53s 125ms/step - loss: 0.2964 - accuracy: 0.8939 - val_loss: 0.3005 - val_accuracy: 0.8915


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▂▁
val_accuracy,▁▃▆▇█
val_loss,█▆▃▂▁
accuracy,0.89394
best_epoch,4
best_val_loss,0.30045
epoch,4
loss,0.29642
val_accuracy,0.8915
